## Part6 RAG-Fusion

- 質問から異なるクエリを生成
- 各クエリで文書を検索
- 検索結果をRRFでランキング
- ランキングされた文書をコンテキストにして回答を生成

In [10]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.prompts import ChatPromptTemplate

template = """あなたは1つの入力クエリに基づいて複数の検索クエリを生成する役立つアシスタントです。

以下に関連する複数の検索クエリを生成してください：{question}

出力（4つのクエリ）:"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)
prompt_rag_fusion

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='あなたは1つの入力クエリに基づいて複数の検索クエリを生成する役立つアシスタントです。\n\n以下に関連する複数の検索クエリを生成してください：{question}\n\n出力（4つのクエリ）:'), additional_kwargs={})])

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [9]:
generate_queries.invoke({"question": "RAGとはなんですか？"})

['1. RAGとは何ですか？', '2. RAGの意味は何ですか？', '3. RAGの略は何ですか？', '4. RAGについて詳しく教えてください。']

In [12]:
results = (generate_queries | retriever.map()).invoke({"question": "RAGとはなんですか？"})
results

[[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGとは。なぜ必要なのか？\nRAGとは、大雑把に言うと、ファイルを参照して回答できるLLM（例えばChatGPT）を作成するための方法です。RAGがなぜ必要なのかというと、通常のLLMでは、回答の正確性を向上させるのに限界があるからです。通常のLLMでは、事実と違う内容を勝手に捏造してしまったり（ハルシネーション）、そもそも学習データに含まれていない情報（例えば公開されていない社内の文書）については、回答することはできなかったりという限界があります。'),
  Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの仕組み\nこの記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。\n「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」\n非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。\n\nRAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...'),
  Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの精度向上のために必要なこと\nRAGの精度向上を試みる際に重要な要素は、以下の2

In [21]:
from langchain.load import dumps, loads


# RRF
# https://www.perplexity.ai/search/reciprocal-rank-fusionnituitej-tUsKo.t4SCWzUFoogZ6dRg
def reciprocal_rank_fusion(results: list[list], k=60):
    """ 複数のランク付けされたドキュメントリストを受け取り、RRF（相互ランクフュージョン）の
        計算に使う任意のパラメータkを使用する関数 """
    
    # ユニークなドキュメントごとの融合されたスコアを保持する辞書を初期化
    fused_scores = {}

    # 各ランク付けされたドキュメントリストを順に処理
    for docs in results:
        # リスト内の各ドキュメントを、そのランク（リスト内での位置）とともに処理
        for rank, doc in enumerate(docs):
            # ドキュメントをキーとして使用するために文字列形式に変換（ドキュメントがJSONにシリアライズできることを前提）
            doc_str = dumps(doc)
            # ドキュメントがまだ辞書に存在しない場合は、初期スコア0で追加
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # ドキュメントの現在のスコアを取得（存在する場合）
            previous_score = fused_scores[doc_str]
            # RRFの公式を使用してスコアを更新: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 融合されたスコアに基づいてドキュメントを降順にソートし、最終的な再ランク付け結果を取得
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # ドキュメントとその融合スコアを含むタプルのリストとして、再ランク付けされた結果を返す
    return reranked_results

In [22]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": "RAGとはなんですか？"})

/tmp/ipykernel_715395/3747162137.py:27: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


In [23]:
docs

[(Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの精度向上のために必要なこと\nRAGの精度向上を試みる際に重要な要素は、以下の2点に分解できます。\n\nユーザーの質問に回答するために最も必要な（最も関連している）ドキュメント群を抽出すること\n抽出してきたドキュメント群を最大限上手く活用して、正しい回答を生成すること\n\nこの2点は、上の「RAGの仕組み」で登場した画像の中の黄色い枠で囲まれている部分に該当します。\n※具体的な手法は、今後の記事で紹介していきます\n\n RAGを使ってできること\n\n個人的には、実務でRAGを使っていて嬉しいポイントは\n\n情報不十分なとき、回答しないことができる\n独自のドキュメントに基づいて回答できる（上の画像には含まれていませんが）'),
  0.06585580821434867),
 (Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの仕組み\nこの記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。\n「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」\n非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。\n\nRAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...'),
  0.06558258417063283),
 (Document(metadata={'

In [27]:
from operator import itemgetter

from networkx import laplacian_matrix

template = """以下のコンテキストに基づいて以下の質問に答えてください。

{context}

質問: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"question": "RAGとはなんですか？"}))

回答: RAGとは、ファイルを参照して回答できるLLM（例えばChatGPT）を作成するための方法であり、通常のLLMでは回答の正確性を向上させるのに限界があるため必要とされる手法です。通常のLLMでは、事実と違う内容を勝手に捏造してしまったり、学習データに含まれていない情報については回答することができないという限界があります。
